In [21]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

In [22]:
print('Pandas : %s'%(pd.__version__))
print('Numpy : %s'%(np.__version__))
print('Scikit-Learn : %s'%(sklearn.__version__))
!python --version

Pandas : 1.0.1
Numpy : 1.18.1
Scikit-Learn : 0.22.1
Python 3.7.6


In [23]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

def grap_main(data) :
    
    retail = ['건강보조식품 소매업', '과실 및 채소 소매업', '관광 민예품 및 선물용품 소매업', '그외 기타 종합 소매업', '기타 대형 종합 소매업', '기타음식료품위주종합소매업', '면세점', '수산물 소매업', '슈퍼마켓', '빵 및 과자류 소매업', '육류 소매업', '차량용 가스 충전업', '차량용 주유소 운영업', '체인화 편의점', '화장품 및 방향제 소매업' ]
    sport_service = ['골프장 운영업', '그외 기타 분류안된 오락관련 서비스업', '그외 기타 스포츠시설 운영업', '기타 수상오락 서비스업']
    personal_service = ['마사지업', '욕탕업']
    accom_rest = ['기타 외국식 음식점업', '기타 주점업', '비알콜 음료점업', '서양식 음식점업', '여관업', '일반유흥 주점업', '일식 음식점업', '중식 음식점업', '피자 햄버거 샌드위치 및 유사 음식점업', '한식 음식점업', '호텔업', '휴양콘도 운영업' ]
    transport = ['내항 여객 운송업', '버스 운송업', '택시 운송업', '정기 항공 운송업']
    rental = ['스포츠 및 레크레이션 용품 임대업', '자동차 임대업']
    business_service = ['여행사업', '전시 및 행사 대행업']
    
    if data in retail :
        return '도매 및 여가'
    elif data in sport_service :
        return '예술, 스포츠 및 여가관련 서비스업'
    elif data in personal_service :
        return '협회 및 단체, 수리 및 기타개인 서비스업'
    elif data in accom_rest :
        return '숙박업 및 음식점업'
    elif data in transport :
        return '운수업'
    elif data in rental :
        return '부동산 및 임대업' 
    elif data in business_service :
        return '사업시설 관리 및 사업지원 서비스업'

In [24]:
# 날짜 처리
data = pd.read_csv('./201901-202003.csv')
data = data.fillna('')
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data['main_category'] = data['STD_CLSS_NM'].apply(lambda x : grap_main(x))
data = data.drop(['REG_YYMM'], axis=1)

In [25]:
data.head()

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,year,month,main_category
0,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4,2019,1,도매 및 여가
1,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8,2019,1,도매 및 여가
2,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6,2019,1,도매 및 여가
3,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5,2019,1,도매 및 여가
4,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3,2019,1,도매 및 여가


In [39]:
# 데이터 정제
df = data.copy()
df = df.drop(['HOM_SIDO_NM', 'HOM_CCG_NM'], axis=1)

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month', 'main_category']
df = df.groupby(columns).sum().reset_index(drop=False)

In [40]:
# 인코딩
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()

for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

In [41]:
# feature, target 설정

train_num = df_num.sample(frac=1, random_state=0)
x = train_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
y = np.log1p(train_num['AMT'])
    
k = int(len(x)*0.9)

In [42]:
x_train = x[:k]
y_train = y[:k]
x_val = x[k:]
y_val = y[k:]

In [43]:
import lightgbm as lgb

In [44]:
train_ds = lgb.Dataset(x_train, label=y_train)
val_ds = lgb.Dataset(x_val, label=y_val)

In [45]:
params = {
            'learning_rate' : 0.05,
            'boosting_type': 'gbdt',
            'objective': 'tweedie',
            'tweedie_variance_power': 1.1,
            'metric': 'rmse',
            'sub_row' : 0.75,
            'lambda_l2' : 0.1
        }

In [46]:
model = lgb.train(params,
                  train_ds,
                  1000,
                  val_ds,
                  verbose_eval = 100,
                  early_stopping_rounds = 100
                 )

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 1.22029
[200]	valid_0's rmse: 0.971414
[300]	valid_0's rmse: 0.876045
[400]	valid_0's rmse: 0.800628
[500]	valid_0's rmse: 0.756365
[600]	valid_0's rmse: 0.721583
[700]	valid_0's rmse: 0.69318
[800]	valid_0's rmse: 0.67003
[900]	valid_0's rmse: 0.652911
[1000]	valid_0's rmse: 0.636874
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 0.636874


In [47]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
main_categorys = df_num['main_category'].unique()
years         = [2020]
months        = [4, 7]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for AGE in AGEs:
            for SEX_CTGO_CD in SEX_CTGO_CDs:
                for FLC in FLCs:
                    for main_category in main_categorys :
                        for year in years:
                            for month in months:
                                temp.append([CARD_SIDO_NM, STD_CLSS_NM, AGE, SEX_CTGO_CD, FLC, year, month, main_category])
                                    
                                    
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=x.columns)

In [48]:
# 예측
pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [49]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [50]:
# 제출 파일 만들기
submission = pd.read_csv('./submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('submission.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,1.466665e+09
1,202004,강원,골프장 운영업,1.452086e+10
2,202004,강원,과실 및 채소 소매업,1.162719e+10
3,202004,강원,관광 민예품 및 선물용품 소매업,4.809192e+08
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,8.820213e+07
